# Adaptive Knowledge System Demo

This notebook demonstrates the basics of the Adaptive Compressed Knowledge Representation System.

In [ ]:
import sys
import os
import asyncio
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# Add the parent directory to Python path to import the package
sys.path.insert(0, os.path.abspath('..'))

In [ ]:
# Import the knowledge system
from src.knowledge.adaptive_knowledge_system import AdaptiveKnowledgeSystem, CompressedContextPack

## Initialize the Knowledge System

In [ ]:
# Create a knowledge system
knowledge_system = AdaptiveKnowledgeSystem()
print("Knowledge system initialized")

## Helper function for async execution in notebook

In [ ]:
# Helper function to run async code in notebook
async def run_async(coro):
    return await coro

def execute_async(coro):
    return asyncio.run(coro)

## Add Knowledge to the System

In [ ]:
# Add some knowledge about AI concepts
async def add_ai_knowledge():
    # Add knowledge about machine learning
    ml_context_id = await knowledge_system.add_knowledge(
        "Machine Learning is a subset of artificial intelligence that involves training algorithms "
        "on data to make predictions or decisions without being explicitly programmed to do so. "
        "Common techniques include neural networks, decision trees, and support vector machines.",
        ["machine learning", "algorithms", "neural networks", "decision trees"]
    )
    
    # Add knowledge about neural networks
    nn_context_id = await knowledge_system.add_knowledge(
        "Neural Networks are a set of algorithms, modeled loosely after the human brain, that "
        "are designed to recognize patterns. They interpret sensory data through a kind of machine "
        "perception, labeling or clustering raw input. They are composed of layers of artificial neurons.",
        ["neural networks", "algorithms", "patterns", "neurons"]
    )
    
    # Add knowledge about deep learning
    dl_context_id = await knowledge_system.add_knowledge(
        "Deep Learning is a subset of machine learning that uses neural networks with multiple layers "
        "(deep neural networks) to analyze various factors of data. Deep learning is behind many recent "
        "advances in AI, including computer vision and natural language processing.",
        ["deep learning", "neural networks", "computer vision", "natural language processing"]
    )
    
    return {
        "machine_learning": ml_context_id,
        "neural_networks": nn_context_id,
        "deep_learning": dl_context_id
    }

context_ids = execute_async(add_ai_knowledge())
print("Added knowledge with context IDs:", context_ids)

## Query the Knowledge System

In [ ]:
# Query for knowledge about neural networks
async def query_knowledge(query_text):
    results = await knowledge_system.query_knowledge(query_text, max_results=3)
    return results

query_results = execute_async(query_knowledge("How do neural networks work?"))

# Display the results
for i, result in enumerate(query_results, 1):
    print(f"Result {i} (Relevance: {result['relevance_score']:.2f})")
    print(f"Summary: {result['summary']}\n")

## Expand Knowledge

In [ ]:
# Expand a specific context
async def expand_context(context_id):
    expanded = await knowledge_system.expand_knowledge(context_id)
    return expanded

nn_expanded = execute_async(expand_context(context_ids["neural_networks"]))

print("Expanded content:")
print(nn_expanded["expanded_content"])

print("\nRelated contexts:")
for related in nn_expanded["related_contexts"]:
    print(f"- {related['summary']} (Relevance: {related['relevance']:.2f})")

## Update Knowledge

In [ ]:
# Update a knowledge context with new information
async def update_knowledge(context_id, new_text):
    updated = await knowledge_system.update_knowledge(context_id, new_text)
    return updated

dl_updated = execute_async(update_knowledge(
    context_ids["deep_learning"],
    "Deep Learning is a subset of machine learning that uses neural networks with multiple layers "
    "(deep neural networks) to analyze various factors of data. Recent advances in deep learning "
    "include transformer models like BERT and GPT, which have revolutionized natural language processing. "
    "Deep learning is also used in computer vision, reinforcement learning, and generative models."
))

print(f"Knowledge updated: {dl_updated}")

# Check the updated content
if dl_updated:
    dl_expanded = execute_async(expand_context(context_ids["deep_learning"]))
    print("\nUpdated content:")
    print(dl_expanded["expanded_content"])

## Visualize Knowledge Graph

In [ ]:
# Create a simplified visualization of the knowledge graph
def visualize_knowledge_graph(knowledge_system):
    # Create a graph
    G = nx.Graph()
    
    # Add nodes for each context
    for node in knowledge_system.context_graph.graph.nodes:
        G.add_node(node, label=node[:8])
    
    # Add edges for relationships
    for u, v, data in knowledge_system.context_graph.graph.edges(data=True):
        G.add_edge(u, v, weight=data.get('weight', 0.5))
    
    # Visualize
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G, seed=42)
    
    # Get edge weights for width
    edge_weights = [G[u][v]['weight'] * 5 for u, v in G.edges()]
    
    # Draw the graph
    nx.draw_networkx_nodes(G, pos, node_size=700, node_color='skyblue', alpha=0.8)
    nx.draw_networkx_edges(G, pos, width=edge_weights, alpha=0.5, edge_color='gray')
    nx.draw_networkx_labels(G, pos, labels={n: G.nodes[n]['label'] for n in G.nodes()})
    
    plt.title("Knowledge Graph Visualization")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize the knowledge graph
visualize_knowledge_graph(knowledge_system)

## Analyze Event-Triggered Updates

In [ ]:
# Test the event-triggered update system
async def test_event_triggering():
    # Create a new context
    context_id = await knowledge_system.add_knowledge(
        "Reinforcement Learning is a type of machine learning where an agent learns to make decisions "
        "by taking actions in an environment to maximize some notion of cumulative reward.",
        ["reinforcement learning", "agent", "reward"]
    )
    
    # Make a small update (shouldn't trigger an update)
    small_update = await knowledge_system.update_knowledge(
        context_id,
        "Reinforcement Learning is a type of machine learning where an agent learns to make decisions "
        "by taking actions in an environment to maximize rewards over time."
    )
    
    # Make a larger update (should trigger an update)
    large_update = await knowledge_system.update_knowledge(
        context_id,
        "Reinforcement Learning is an area of machine learning concerned with how intelligent agents "
        "ought to take actions in an environment in order to maximize the notion of cumulative reward. "
        "Key algorithms include Q-learning, SARSA, and Deep Q Networks (DQN). RL has been applied to "
        "robotics, game playing, and autonomous vehicles."
    )
    
    return {
        "context_id": context_id,
        "small_update_triggered": small_update,
        "large_update_triggered": large_update
    }

event_results = execute_async(test_event_triggering())

print(f"Event-Triggered Update Test Results:")
print(f"Context ID: {event_results['context_id']}")
print(f"Small update triggered: {event_results['small_update_triggered']}")
print(f"Large update triggered: {event_results['large_update_triggered']}")

## Exploring Threshold Adaptation

In [ ]:
# Extract threshold history
def analyze_thresholds(knowledge_system):
    event_trigger = knowledge_system.event_trigger
    history = event_trigger.event_history
    
    if not history:
        print("No event history available")
        return
    
    # Group by context ID
    contexts = {}
    for event in history:
        context_id = event['context_id']
        if context_id not in contexts:
            contexts[context_id] = []
        contexts[context_id].append(event)
    
    # Plot threshold adaptation for the context with the most events
    most_active_context = max(contexts.items(), key=lambda x: len(x[1]))
    context_id, events = most_active_context
    
    print(f"Analyzing threshold adaptation for context {context_id}")
    print(f"Total events: {len(events)}")
    
    # Extract data for plotting
    timestamps = [e['timestamp'] for e in events]
    magnitudes = [e['change_magnitude'] for e in events]
    triggered = [e['was_triggered'] for e in events]
    
    # Convert timestamps to relative time
    start_time = min(timestamps)
    relative_times = [(t - start_time) for t in timestamps]
    
    # Plot the results
    plt.figure(figsize=(12, 6))
    
    # Plot change magnitudes
    plt.scatter(relative_times, magnitudes, c=['green' if t else 'red' for t in triggered], 
                alpha=0.7, s=100, label='Change Magnitude')
    
    # Add legend and labels
    plt.axhline(y=event_trigger.get_threshold(context_id), color='blue', linestyle='--', 
                label=f'Current Threshold: {event_trigger.get_threshold(context_id):.3f}')
    plt.xlabel('Relative Time (seconds)')
    plt.ylabel('Change Magnitude')
    plt.title('Event Triggering Analysis')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

analyze_thresholds(knowledge_system)

## Conclusion

This notebook demonstrates the core functionality of the Adaptive Compressed Knowledge Representation System:

1. Adding knowledge to the system
2. Querying for relevant information
3. Expanding context to get detailed information
4. Updating knowledge with new information
5. Visualizing the knowledge graph
6. Testing the event-triggered update system
7. Analyzing threshold adaptation

These capabilities form the foundation of the Adaptive Compressed World Model Framework, allowing for efficient management of knowledge in memory and storage while maintaining the ability to retrieve detailed information when needed.